In [1]:
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
import numpy as np
import catboost
from catboost import Pool, CatBoostRegressor
print(catboost.__version__)

1.0.3


In [5]:
df = pd.read_csv('data/data_to_boosting.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)

In [6]:
'''
'*num*w_2' ...  -> медианные значения за последние 3 месяца до *num* месяца (не включая)
'''

"\n'*num*w_2' ...  -> медианные значения за последние 3 месяца до *num* месяца (не включая)\n"

In [7]:
# Выборка фич для трейна на 9 и предикт 10
col_to_train_X = ['prod_norm_name', 'prod_form_norm_name', 'code', 'prod_pack_1_2',
                         'prod_pack_1_size', 'origin', 'ЖНВЛП', 'Ковид',
                         '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', 
                         '2_conf', '3_conf', '4_conf', '5_conf', '6_conf', '7_conf',
                         '8_conf', '2_covd', '3_covd', '4_covd',
                         '5_covd', '6_covd', '7_covd', '8_covd', 
                         '2_covi', '3_covi', '4_covi', '5_covi', '6_covi', '7_covi', '8_covi',
                         '2_caseF', '3_caseF', '4_caseF',
                         '5_caseF', '6_caseF', '7_caseF', '8_caseF',
                         '5.0w_1', '6.0w_2', '7.0w_3', '8.0w_4', '9.0w_5']

col_to_train_Y = ['9.0']

col_to_test_X = ['prod_norm_name', 'prod_form_norm_name', 'code', 'prod_pack_1_2',
                  'prod_pack_1_size', 'origin', 'ЖНВЛП', 'Ковид',
                  '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0',
                  '3_conf', '4_conf', '5_conf', '6_conf', '7_conf',
                  '8_conf', '9_conf', '3_covd', '4_covd',
                  '5_covd', '6_covd', '7_covd', '8_covd', '9_covd',  
                  '3_covi', '4_covi', '5_covi', '6_covi', '7_covi', '8_covi',
                  '9_covi', '3_caseF', '4_caseF',
                  '5_caseF', '6_caseF', '7_caseF', '8_caseF', '9_caseF', 
                  '6.0w_2', '7.0w_3', '8.0w_4', '9.0w_5', '10.0w_6']

col_to_test_Y = ['10.0']

print(f'Last train mounth: {col_to_train_Y[-1]}')
print(f'Last test mounth: {col_to_test_Y[-1]}')
print('Train features', len(col_to_train_X))
print('Test features', len(col_to_test_X))

Last train mounth: 9.0
Last test mounth: 10.0
Train features 48
Test features 48


In [8]:
# Выборка фич для трейна на 8 и предикт 9
col_to_train_X = ['prod_norm_name', 'prod_form_norm_name', 'code', 'prod_pack_1_2',
                         'prod_pack_1_size', 'origin', 'ЖНВЛП', 'Ковид', 
                         '1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0',
                         '1_conf', '2_conf', '3_conf', '4_conf', '5_conf', '6_conf', '7_conf',
                         '1_covd', '2_covd', '3_covd', '4_covd',
                         '5_covd', '6_covd', '7_covd', '1_covi',
                         '2_covi', '3_covi', '4_covi', '5_covi', '6_covi', '7_covi',
                         '1_caseF', '2_caseF', '3_caseF', '4_caseF',
                         '5_caseF', '6_caseF', '7_caseF',
                         '4.0w_0', '5.0w_1', '6.0w_2', '7.0w_3', '8.0w_4']

col_to_train_Y = ['8.0']

col_to_test_X = ['prod_norm_name', 'prod_form_norm_name', 'code', 'prod_pack_1_2',
                  'prod_pack_1_size', 'origin', 'ЖНВЛП', 'Ковид', 
                  '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', 
                  '2_conf', '3_conf', '4_conf', '5_conf', '6_conf', '7_conf',
                  '8_conf', '2_covd', '3_covd', '4_covd',
                  '5_covd', '6_covd', '7_covd', '8_covd', 
                  '2_covi', '3_covi', '4_covi', '5_covi', '6_covi', '7_covi', '8_covi',
                  '2_caseF', '3_caseF', '4_caseF',
                  '5_caseF', '6_caseF', '7_caseF', '8_caseF',
                  '5.0w_1', '6.0w_2', '7.0w_3', '8.0w_4', '9.0w_5']

col_to_test_Y = ['9.0']

print(f'Last train mounth: {col_to_train_Y[-1]}')
print(f'Last test mounth: {col_to_test_Y[-1]}')
print('Train features', len(col_to_train_X))
print('Test features', len(col_to_test_X))

Last train mounth: 8.0
Last test mounth: 9.0
Train features 48
Test features 48


In [9]:
# 1_smt - первый с конца (самый старый месяц)
feature_col_X = ['prod_norm_name', 'prod_form_norm_name', 'code', 'prod_pack_1_2',
               'prod_pack_1_size', 'origin', 'ЖНВЛП', 'Ковид', 
               '1_mounth', '2_mounth', '3_mounth', '4_mounth', '5_mounth', '6_mounth', '7_mounth', 
               '1_conf', '2_conf', '3_conf', '4_conf', '5_conf', '6_conf', '7_conf',
               '1_covd', '2_covd', '3_covd', '4_covd', '5_covd', '6_covd', '7_covd', 
               '1_covi', '2_covi', '3_covi', '4_covi', '5_covi', '6_covi', '7_covi',
               '1_caseF', '2_caseF', '3_caseF','4_caseF', '5_caseF', '6_caseF', '7_caseF',
               '1_w', '2_w', '3_w', '4_w', '5_w']
               
feature_col_Y = ['target']
print(len(feature_col_X))

48


In [10]:
mapping_name = pd.DataFrame(df.prod_form_norm_name.unique()).reset_index().set_index(0).to_dict()['index']

In [11]:
df.prod_form_norm_name = df.prod_form_norm_name.map(mapping_name)

In [12]:
# сейчас трейним на 8м и предиктим 9й
train_X = df.loc[:, col_to_train_X]
train_X.columns = feature_col_X
train_Y = df.loc[:, col_to_train_Y]
train_Y.columns = feature_col_Y
test_X = df.loc[:, col_to_test_X]
test_X.columns = feature_col_X
test_Y = df.loc[:, col_to_test_Y]
test_Y.columns = feature_col_Y

In [13]:
# логарифмируем целевую переменную
test_X.loc[:, ['1_mounth', '2_mounth', '3_mounth', '4_mounth', '5_mounth', '6_mounth', '7_mounth']] = (test_X.loc[:, ['1_mounth', '2_mounth', '3_mounth', '4_mounth', '5_mounth', '6_mounth', '7_mounth']] + 1).apply(np.log)
train_X.loc[:, ['1_mounth', '2_mounth', '3_mounth', '4_mounth', '5_mounth', '6_mounth', '7_mounth']] = (test_X.loc[:, ['1_mounth', '2_mounth', '3_mounth', '4_mounth', '5_mounth', '6_mounth', '7_mounth']] + 1).apply(np.log)
train_Y = (train_Y + 1).apply(np.log)
test_Y = (test_Y + 1).apply(np.log)

In [14]:
# приводим к нужному виду
test_X['origin'] = test_X['origin'].astype(int)
test_X['prod_pack_1_2'] = test_X['prod_pack_1_2'].astype(int)
test_X['code'] = test_X['code'].astype(int)
test_X['prod_norm_name'] = test_X['prod_norm_name'].astype(int)

train_X['origin'] = train_X['origin'].astype(int)
train_X['prod_pack_1_2'] = train_X['prod_pack_1_2'].astype(int)
train_X['code'] = train_X['code'].astype(int)
train_X['prod_norm_name'] = train_X['prod_norm_name'].astype(int)

In [15]:
cat_features = ['prod_norm_name', 'code', 'prod_pack_1_2', 
                'origin', 'ЖНВЛП', 'Ковид', 'prod_form_norm_name']


In [32]:
train_pool = Pool(
    train_X, train_Y, 
    cat_features=cat_features,
)

validation_pool = Pool(
    test_X, test_Y, 
    cat_features=cat_features,
)

In [42]:
model = CatBoostRegressor(
    iterations=200,
    learning_rate=0.001,
    depth=4,
    task_type="GPU",
    random_seed=1
)

model.fit(
    train_pool,
    eval_set=validation_pool,
    verbose=50
);

0:	learn: 1.6083980	test: 0.5632409	best: 0.5632409 (0)	total: 13.5ms	remaining: 2.69s
50:	learn: 1.5379386	test: 0.4880806	best: 0.4880806 (50)	total: 769ms	remaining: 2.25s
100:	learn: 1.4709752	test: 0.4165026	best: 0.4165026 (100)	total: 1.56s	remaining: 1.53s
150:	learn: 1.4069714	test: 0.3624589	best: 0.3624589 (150)	total: 2.36s	remaining: 767ms
199:	learn: 1.3472216	test: 0.3335525	best: 0.3335525 (199)	total: 3.03s	remaining: 0us
bestTest = 0.3335524618
bestIteration = 199


In [43]:
# 9 10
prediction = model.predict(test_X)
print('MAE:', mean_absolute_error(np.exp(prediction) - 1, test_Y.apply(np.exp) - 1))
print('MAPE:',mean_absolute_percentage_error(np.exp(prediction) - 1, test_Y.apply(np.exp) - 1))

MAE: 172.62896006903264
MAPE: 0.3180373128320467


In [44]:
# model.save_model('model_2.py', format='python')